In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report 
%matplotlib inline

In [2]:
w, h, c = 256, 256, 3
m_train = 2682
m_test = 200


train_dir = 'dataset/train'
test_dir = 'dataset/test'

shapes_addr_tr = os.listdir(train_dir)
shapes_addr_ts = os.listdir(test_dir)

shapes_abs_addresses_tr = [train_dir + '/' + each_shape_dir for each_shape_dir in shapes_addr_tr]
shapes_abs_addresses_ts = [test_dir + '/' + each_shape_dir for each_shape_dir in shapes_addr_ts]

num_train_items = {k:len(os.listdir(each_item)) for (k, each_item) in zip(shapes_addr_tr, shapes_abs_addresses_tr)}
num_test_items = {k:len(os.listdir(each_item)) for (k, each_item) in zip(shapes_addr_ts, shapes_abs_addresses_ts)}

In [3]:
def load_data(dir, m):
    
    tmp = 0
    imgs_dirs = os.listdir(dir)
    imgs_dirs = [dir + '/' + each for each in imgs_dirs]
    arr = np.zeros((m, 256, 256, 3), dtype='uint8')
    
    for each_img_dir in imgs_dirs:
        arr[tmp] = cv2.GaussianBlur(cv2.imread(each_img_dir), (5, 5), 0, 0)
        tmp += 1
        
    return arr

In [4]:
train_arr_circle = load_data('dataset/train/circle', num_train_items['circle'])
train_arr_heart = load_data('dataset/train/heart', num_train_items['heart'])
train_arr_square = load_data('dataset/train/square', num_train_items['square'])
train_arr_star = load_data('dataset/train/star', num_train_items['star'])
train_arr_triangle = load_data('dataset/train/triangle', num_train_items['triangle'])

tr_imgs = [train_arr_circle, train_arr_heart, train_arr_square, train_arr_star, train_arr_triangle]


test_arr_circle = load_data('dataset/test/circle', num_test_items['circle'])
test_arr_heart = load_data('dataset/test/heart', num_test_items['heart'])
test_arr_square = load_data('dataset/test/square', num_test_items['square'])
test_arr_star = load_data('dataset/test/star', num_test_items['star'])
test_arr_triangle = load_data('dataset/test/triangle', num_test_items['triangle'])

ts_imgs = [test_arr_circle, test_arr_heart, test_arr_square, test_arr_star, test_arr_triangle]

In [5]:
def data_process(arr, num_items, shape):
    
    m, w, h, c = arr.shape
    edge_detected_arr = np.zeros((m, w, h), dtype='uint8')
    feature_extracted_list = []
    orb = cv2.ORB_create()
    defects = 0
    
    for i in range(m):
        edge_detected_arr[i] = cv2.Canny(image=arr[i], threshold1=100, threshold2=250)

    
        _, tmp = orb.detectAndCompute(edge_detected_arr[i], None)
        
        if tmp is not None:
            feature_extracted_list.append(tmp)
        else:
            defects += 1
            continue

    
    num_items[shape] = num_items[shape] - defects
    
    return feature_extracted_list, num_items
    

In [6]:
processed_tr_circle, num_train_items   = data_process(train_arr_circle, num_train_items, 'circle')
processed_tr_heart, num_train_items    = data_process(train_arr_heart, num_train_items, 'heart')
processed_tr_square, num_train_items   = data_process(train_arr_square, num_train_items, 'square')
processed_tr_star, num_train_items     = data_process(train_arr_star, num_train_items, 'star')
processed_tr_triangle, num_train_items = data_process(train_arr_triangle, num_train_items, 'triangle')

features_tr = processed_tr_circle + processed_tr_heart + processed_tr_square + processed_tr_star + processed_tr_triangle


processed_ts_circle, num_test_items   = data_process(ts_imgs[0], num_test_items, 'circle')
processed_ts_heart, num_test_items    = data_process(ts_imgs[1], num_test_items, 'heart')
processed_ts_square, num_test_items   = data_process(ts_imgs[2], num_test_items, 'square')
processed_ts_star, num_test_items     = data_process(ts_imgs[3], num_test_items, 'star')
processed_ts_triangle, num_test_items = data_process(ts_imgs[4], num_test_items, 'triangle')

features_ts = processed_ts_circle + processed_ts_heart + processed_ts_square + processed_ts_star + processed_ts_triangle

In [7]:
def BF_FeatureMatcher(des1,des2):
    brute_force = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    no_of_matches = brute_force.match(des1,des2)
 
    no_of_matches = sorted(no_of_matches,key=lambda x:x.distance)
    return len(no_of_matches)

In [8]:
y, y_pred = [], []
num_items = list(num_train_items.values())

for i in range(len(features_ts)):
    
    tmp = []
    averages = []
    for j in range(len(features_tr)):
        tmp.append(BF_FeatureMatcher(features_tr[j], features_ts[i]))
        
    averages.append(sum(tmp[0:num_items[0]]) / num_items[0])
    averages.append(sum(tmp[num_items[0]: num_items[0] + num_items[1]]) / num_items[1])
    
    averages.append(sum(tmp[num_items[0] + num_items[1]: num_items[0] + num_items[1] + num_items[2]]) / num_items[2])
    
    averages.append(sum(tmp[num_items[0] + num_items[1] + num_items[2]: num_items[0] + num_items[1] + num_items[2] + num_items[3]]) / num_items[3])
    averages.append(sum(tmp[num_items[0] + num_items[1] + num_items[2] + num_items[3]: num_items[0] + num_items[1] + num_items[2] + num_items[3] + num_items[4]]) / num_items[4])

    print('sample: ', str(i))
    print('average predictions: ', averages)
    print()
    y_pred.append(averages.index(max(averages)))
    y.append(i // 40)
        

sample:  0
average predictions:  [73.76421404682274, 58.03249097472924, 56.19216417910448, 59.690298507462686, 62.09251101321586]

sample:  1
average predictions:  [67.623745819398, 38.68411552346571, 46.86194029850746, 40.62313432835821, 47.0352422907489]

sample:  2
average predictions:  [74.0, 55.16245487364621, 57.36940298507463, 60.49813432835821, 59.10132158590309]

sample:  3
average predictions:  [73.88461538461539, 53.041516245487365, 55.15485074626866, 54.832089552238806, 58.94493392070485]

sample:  4
average predictions:  [67.43812709030101, 36.80324909747292, 44.45335820895522, 37.42910447761194, 44.852422907488986]

sample:  5
average predictions:  [75.7809364548495, 58.871841155234655, 58.17537313432836, 62.17164179104478, 63.05726872246696]

sample:  6
average predictions:  [76.93812709030101, 62.935018050541515, 58.72201492537314, 62.74626865671642, 66.16519823788546]

sample:  7
average predictions:  [72.60033444816054, 54.52707581227437, 54.58955223880597, 58.1138059

In [9]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.98      1.00      0.99        40
           2       1.00      0.38      0.55        40
           3       0.95      1.00      0.98        40
           4       0.61      0.95      0.75        40

    accuracy                           0.86       200
   macro avg       0.91      0.86      0.85       200
weighted avg       0.91      0.86      0.85       200

